In [17]:
import pandas as pd
import pytest
import pickle
from flu_ani_analysis import flu_ani_analysis as fa

def test_read_in_envision():
    
    # read in expected data from .pkl file
    with open("C:\\Users\\Bartek\\Documents\\MChem project\\tests\\example outputs\\plate_1_repeat_test.pkl", 'rb') as file:
        expected_output = pickle.load(file)
    
    # execute the tested function
    actual_dict = fa.FA.read_in_envision(csv_file = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1_repeat.csv", data_type='plate', size=384)
    
    #convert the dict into a list of data frames
    actual_list = []
    for rep in actual_dict.data_dict.values():
        m, d = rep.values()
        p, s = d.values()
        actual_list.append(m)
        actual_list.append(p)
        actual_list.append(s)
    
    # compare the expected and actual data frames
    for output, example in zip(expected_output, actual_list):
        pd.testing.assert_frame_equal(output, example)